In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
from pathlib import Path

IMAGES_PATH = Path() / "images" / "College plots"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [6]:
data = pd.read_csv('top_colleges_2022.csv')
data.head(3)

,description,rank,organizationName,state,studentPopulation,campusSetting,medianBaseSalary,longitude,latitude,website,...,yearFounded,stateCode,collegeType,carnegieClassification,studentFacultyRatio,totalStudentPop,undergradPop,totalGrantAid,percentOfStudentsFinAid,percentOfStudentsGrant
0,"A leading global research university, MIT attr...",1,Massachusetts Institute of Technology,MA,12195,Urban,173700.0,-71.093539,42.359006,http://web.mit.edu,...,1861.0,MA,Private not-for-profit,Doctoral Universities: Very High Research Acti...,3,12195,4582,35299332.0,75.0,60.0
1,Stanford University sits just outside of Palo ...,2,Stanford University,CA,20961,Suburban,173500.0,-122.168924,37.431370,http://www.stanford.edu,...,1891.0,CA,Private not-for-profit,Doctoral Universities: Very High Research Acti...,4,20961,8464,51328461.0,70.0,55.0
2,One of the top public universities in the coun...,2,"University of California, Berkeley",CA,45878,Urban,154500.0,-122.258393,37.869236,http://www.berkeley.edu,...,1868.0,CA,Public,Doctoral Universities: Very High Research Acti...,19,45878,33208,64495611.0,63.0,53.0


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   description              498 non-null    object 
 1   rank                     498 non-null    int64  
 2   organizationName         498 non-null    object 
 3   state                    498 non-null    object 
 4   studentPopulation        498 non-null    int64  
 5   campusSetting            498 non-null    object 
 6   medianBaseSalary         491 non-null    float64
 7   longitude                458 non-null    float64
 8   latitude                 458 non-null    float64
 9   website                  477 non-null    object 
 10  phoneNumber              428 non-null    object 
 11  city                     498 non-null    object 
 12  country                  498 non-null    object 
 13  state.1                  498 non-null    object 
 14  region                   4

In [11]:
data.nunique()

description                498
rank                       491
organizationName           498
state                       51
studentPopulation          492
campusSetting                3
medianBaseSalary           324
longitude                  455
latitude                   455
website                    477
phoneNumber                428
city                       374
country                      1
state.1                     51
region                       4
yearFounded                175
stateCode                   51
collegeType                  2
carnegieClassification      12
studentFacultyRatio         29
totalStudentPop            492
undergradPop               489
totalGrantAid              495
percentOfStudentsFinAid     56
percentOfStudentsGrant      67
dtype: int64

looking off the bat we see that all colleges are in the US so there is no need for that column. We will drop is later on. it looks like that there is also only 2 types of college, 2 carnegieClassification, and 3 types of campus settings 

In [ ]:
type_and_setting 